# Weather prediction system producer

In [1]:
from kafka import KafkaProducer
from time import sleep
from sodapy import Socrata
import logging
from json import dumps, loads
import csv
from io import StringIO


In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
topic = 'TestDoAn'
producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda K:dumps(K).encode('utf-8'))

INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup


In [4]:
user_name = "<user_name>"
password = "<password>"
api_key = "<api_key>"
client = Socrata('data.cityofchicago.org', api_key,username=user_name,password=password)

In [ ]:
temp = []
while (True):
    try:
        results = client.get("k7hf-8y75", limit=13, where="station_name == 'Oak Street Weather Station'", order ="measurement_timestamp DESC")
        items = [record for record in results if record not in temp]
        for item in items:
            output = StringIO()
            writer = csv.writer(output)
            writer.writerow(item.values())
            csv_string = output.getvalue()
            print(csv_string)
            producer.send(topic, csv_string)
            sleep(1)
        temp = results
        sleep(60)
    except KeyboardInterrupt:
        print('break')
        break
output.close()

INFO:kafka.conn:<BrokerConnection node_id=0 host=LAPTOP-0J6HSUP4:9092 <connecting> [IPv6 ('fe80::c910:fe8d:605b:4633', 9092, 0, 18)]>: connecting to LAPTOP-0J6HSUP4:9092 [('fe80::c910:fe8d:605b:4633', 9092, 0, 18) IPv6]
INFO:kafka.conn:<BrokerConnection node_id=0 host=LAPTOP-0J6HSUP4:9092 <connecting> [IPv6 ('fe80::c910:fe8d:605b:4633', 9092, 0, 18)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Closing connection. 


Oak Street Weather Station,2024-02-01T06:00:00.000,5,3.3,77,0,0,32.2,0,273,3.4,6.9,992,1,358,11.9,02/01/2024 6:00 AM,OakStreetWeatherStation202402010600

Oak Street Weather Station,2024-02-01T05:00:00.000,5.3,3.5,76,0,0,32.2,0,296,3,6.5,992.4,1,358,11.9,02/01/2024 5:00 AM,OakStreetWeatherStation202402010500

Oak Street Weather Station,2024-02-01T04:00:00.000,5,3.4,78,0,0,32.2,0,275,5,8.7,992.6,1,358,11.7,02/01/2024 4:00 AM,OakStreetWeatherStation202402010400

Oak Street Weather Station,2024-02-01T03:00:00.000,5.1,3.5,78,0,0,32.2,0,284,4.5,9.1,992.6,1,358,11.9,02/01/2024 3:00 AM,OakStreetWeatherStation202402010300

Oak Street Weather Station,2024-02-01T02:00:00.000,5,3.5,79,0,0,32.2,0,270,4.3,10.2,993.3,1,358,11.7,02/01/2024 2:00 AM,OakStreetWeatherStation202402010200

Oak Street Weather Station,2024-02-01T01:00:00.000,4.9,3.5,80,0,0,32.2,0,274,4.8,10,993.4,0,358,11.9,02/01/2024 1:00 AM,OakStreetWeatherStation202402010100

Oak Street Weather Station,2024-02-01T00:00:00.000,4.7,3.4,81,0,